In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv("C:/Users/Christopher/Desktop/freecodecamp_algotrade/starter_files/sp_500_stocks.csv")
stocks
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol="AAPL"
api_call =f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_call).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2312480145606,
 'week52high': 151.44,
 'week52low': 87.36,
 'week52highSplitAdjustOnly': 148.91,
 'week52lowSplitAdjustOnly': 90.58,
 'week52change': 0.503841884719005,
 'sharesOutstanding': 17441366641,
 'float': 0,
 'avg10Volume': 89808062,
 'avg30Volume': 75709100,
 'day200MovingAvg': 131.38,
 'day50MovingAvg': 129.54,
 'employees': 151953,
 'ttmEPS': 4.56,
 'ttmDividendRate': 0.8587248269010862,
 'dividendYield': 0.006574456245057594,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-03',
 'nextEarningsDate': '2021-07-18',
 'peRatio': 30.11011548114096,
 'beta': 1.5669301729256035,
 'maxChangePercent': 52.19177216030133,
 'year5ChangePercent': 4.947050263928243,
 'year2ChangePercent': 1.748477952828341,
 'year1ChangePercent': 0.5256157990693248,
 'ytdChangePercent': 0.00025143645059255693,
 'month6ChangePercent': 0.035897842467042425,
 'month3ChangePercent': 0.1093636442185122,
 'month1ChangePercent': 0.05577787109485816,
 'day30ChangeP

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks["Ticker"],100))
symbol_strings= []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [5]:
my_columns = ["Ticker","Price","1 yr Price-Return","3M Price-Return","1M Price-Return","No. of shares to buy"]
final_dataframe = pd.DataFrame(columns = my_columns)

In [6]:
for symbol_string in symbol_strings:
    bsl_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(bsl_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe= final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["price"],
                data[symbol]["stats"]["year1ChangePercent"],
                data[symbol]["stats"]["month3ChangePercent"],
                data[symbol]["stats"]["month1ChangePercent"],
                "N/A"
            ],
            index=my_columns
            ), 
            ignore_index = True
            )
final_dataframe

,Ticker,Price,1 yr Price-Return,3M Price-Return,1M Price-Return,No. of shares to buy
0,A,150.24,0.665372,0.197455,0.111401,N/A
1,AAL,23.54,0.403192,-0.102733,-0.00543208,N/A
2,AAP,202.76,0.388188,0.0739272,0.0239934,N/A
3,AAPL,138.60,0.54435,0.109033,0.0574669,N/A
4,ABBV,120.11,0.252827,0.12787,-0.0125367,N/A
...,...,...,...,...,...,...
500,YUM,119.87,0.362075,0.0892156,-0.0210853,N/A
501,ZBH,166.54,0.315765,0.0226098,-0.0230266,N/A
502,ZBRA,514.45,0.923972,0.0977643,0.02067,N/A
503,ZION,55.23,0.50727,-0.0629601,-0.094633,N/A


In [7]:
final_dataframe.sort_values("1 yr Price-Return", ascending = False, inplace=True)
final_dataframe= final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,1 yr Price-Return,3M Price-Return,1M Price-Return,No. of shares to buy
0,275,LB,68.67,3.69071,0.0921645,0.017964,N/A
1,179,FCX,36.04,2.50821,0.0252691,-0.124792,N/A
2,441,TPR,42.96,2.18207,-0.0587976,-0.017235,N/A
3,208,GPS,32.15,1.99389,0.0287052,-0.0299396,N/A
4,253,IVZ,27.40,1.5475,0.0645054,0.000763717,N/A
5,410,SIVB,565.09,1.54262,0.016737,-0.0298865,N/A
6,233,HWM,34.45,1.46701,0.0707711,0.00929035,N/A
7,149,DXC,38.60,1.44297,0.355648,0.00974404,N/A
8,314,MOS,31.09,1.44292,-0.0758349,-0.131236,N/A
9,272,KSS,53.65,1.43354,-0.13008,-0.00475344,N/A


Enter value of portfolio: 10000000000
portfolio_size is: 10000000000


In [9]:
portfolio_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,["No. of shares to buy"]]= math.floor(portfolio_size/final_dataframe.loc[i,"Price"])
final_dataframe

C:\Users\Christopher\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,1 yr Price-Return,3M Price-Return,1M Price-Return,No. of shares to buy
0,275,LB,68.67,3.69071,0.0921645,0.017964,2912479
1,179,FCX,36.04,2.50821,0.0252691,-0.124792,5549389
2,441,TPR,42.96,2.18207,-0.0587976,-0.017235,4655493
3,208,GPS,32.15,1.99389,0.0287052,-0.0299396,6220839
4,253,IVZ,27.40,1.5475,0.0645054,0.000763717,7299270
5,410,SIVB,565.09,1.54262,0.016737,-0.0298865,353925
6,233,HWM,34.45,1.46701,0.0707711,0.00929035,5805515
7,149,DXC,38.60,1.44297,0.355648,0.00974404,5181347
8,314,MOS,31.09,1.44292,-0.0758349,-0.131236,6432936
9,272,KSS,53.65,1.43354,-0.13008,-0.00475344,3727865


In [20]:
hqm_columns= [
    "Ticker",
    "Price",
    "No of shares to Buy",
    "One-Year Price-Return",
    "One-Year Return Percentile",
    "Six-Month Price-Return",
    "Six-Month Return Percentile",
    "Three-Month Price-Return",
    "Three-Month Return Percentile",
    "One-Month Price-Return",
    "One-Month Return Percentile",    
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)


In [21]:
convert_none = lambda x : 0 if x is None else x
for symbol_string in symbol_strings:
    bsl_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(bsl_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe= hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["price"],
                "N/A",
                convert_none(data[symbol]["stats"]["year1ChangePercent"]),
                "N/A",
                convert_none(data[symbol]["stats"]["month6ChangePercent"]),
                "N/A",
                convert_none(data[symbol]["stats"]["month3ChangePercent"]),
                "N/A",
                convert_none(data[symbol]["stats"]["month1ChangePercent"]),
                "N/A"
            ],
            index=hqm_columns
            ), 
            ignore_index = True
            )
hqm_dataframe


,Ticker,Price,No of shares to Buy,One-Year Price-Return,One-Year Return Percentile,Six-Month Price-Return,Six-Month Return Percentile,Three-Month Price-Return,Three-Month Return Percentile,One-Month Price-Return,One-Month Return Percentile
0,A,152.81,N/A,0.670615,N/A,0.259192,N/A,0.202172,N/A,0.109977,N/A
1,AAL,23.56,N/A,0.412937,N/A,0.409164,N/A,-0.103080,N/A,-0.005532,N/A
2,AAP,200.77,N/A,0.400424,N/A,0.249863,N/A,0.073959,N/A,0.023563,N/A
3,AAPL,137.70,N/A,0.544730,N/A,0.035120,N/A,0.107721,N/A,0.056542,N/A
4,ABBV,118.59,N/A,0.255330,N/A,0.136344,N/A,0.125174,N/A,-0.012004,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.35,N/A,0.358315,N/A,0.085491,N/A,0.089032,N/A,-0.021603,N/A
501,ZBH,165.98,N/A,0.304613,N/A,0.104745,N/A,0.022792,N/A,-0.023778,N/A
502,ZBRA,525.94,N/A,0.908853,N/A,0.348008,N/A,0.099779,N/A,0.020127,N/A
503,ZION,54.81,N/A,0.526726,N/A,0.263427,N/A,-0.063645,N/A,-0.097734,N/A


In [12]:
time_periods = [
            "One-Year",
            "Six-Month",
            "Three-Month",
            "One-Month"
                ]

In [22]:
for row in hqm_dataframe.index:
    for time in time_periods:
        change_col = f"{time} Price-Return"
        percentile_col = f"{time} Return Percentile"
        hqm_dataframe.loc[row , percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
hqm_dataframe

,Ticker,Price,No of shares to Buy,One-Year Price-Return,One-Year Return Percentile,Six-Month Price-Return,Six-Month Return Percentile,Three-Month Price-Return,Three-Month Return Percentile,One-Month Price-Return,One-Month Return Percentile
0,A,152.81,N/A,0.670615,76.2376,0.259192,67.5248,0.202172,88.7129,0.109977,94.6535
1,AAL,23.56,N/A,0.412937,49.901,0.409164,88.9109,-0.103080,4.35644,-0.005532,49.3069
2,AAP,200.77,N/A,0.400424,47.5248,0.249863,65.1485,0.073959,53.0693,0.023563,68.9109
3,AAPL,137.70,N/A,0.544730,63.5644,0.035120,15.8416,0.107721,67.5248,0.056542,82.1782
4,ABBV,118.59,N/A,0.255330,28.5149,0.136344,39.2079,0.125174,71.8812,-0.012004,45.7426
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.35,N/A,0.358315,41.3861,0.085491,26.3366,0.089032,60.198,-0.021603,40.198
501,ZBH,165.98,N/A,0.304613,34.6535,0.104745,33.0693,0.022792,32.8713,-0.023778,38.0198
502,ZBRA,525.94,N/A,0.908853,88.5149,0.348008,82.5743,0.099779,64.3564,0.020127,67.7228
503,ZION,54.81,N/A,0.526726,61.1881,0.263427,68.3168,-0.063645,9.30693,-0.097734,3.56436


In [23]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f"{time} Return Percentile"])
    hqm_dataframe.loc[row , "HQM Score"] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,No of shares to Buy,One-Year Price-Return,One-Year Return Percentile,Six-Month Price-Return,Six-Month Return Percentile,Three-Month Price-Return,Three-Month Return Percentile,One-Month Price-Return,One-Month Return Percentile,HQM Score
0,A,152.81,N/A,0.670615,76.2376,0.259192,67.5248,0.202172,88.7129,0.109977,94.6535,81.782178
1,AAL,23.56,N/A,0.412937,49.901,0.409164,88.9109,-0.103080,4.35644,-0.005532,49.3069,48.118812
2,AAP,200.77,N/A,0.400424,47.5248,0.249863,65.1485,0.073959,53.0693,0.023563,68.9109,58.663366
3,AAPL,137.70,N/A,0.544730,63.5644,0.035120,15.8416,0.107721,67.5248,0.056542,82.1782,57.277228
4,ABBV,118.59,N/A,0.255330,28.5149,0.136344,39.2079,0.125174,71.8812,-0.012004,45.7426,46.336634
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.35,N/A,0.358315,41.3861,0.085491,26.3366,0.089032,60.198,-0.021603,40.198,42.029703
501,ZBH,165.98,N/A,0.304613,34.6535,0.104745,33.0693,0.022792,32.8713,-0.023778,38.0198,34.653465
502,ZBRA,525.94,N/A,0.908853,88.5149,0.348008,82.5743,0.099779,64.3564,0.020127,67.7228,75.792079
503,ZION,54.81,N/A,0.526726,61.1881,0.263427,68.3168,-0.063645,9.30693,-0.097734,3.56436,35.594059


In [29]:
hqm_dataframe.sort_values("HQM Score", ascending = False, inplace= True)
hqm_dataframe[:20]

,Ticker,Price,No of shares to Buy,One-Year Price-Return,One-Year Return Percentile,Six-Month Price-Return,Six-Month Return Percentile,Three-Month Price-Return,Three-Month Return Percentile,One-Month Price-Return,One-Month Return Percentile,HQM Score
148,DVN,29.90,N/A,1.333371,97.2277,0.897961,99.604,0.274373,95.2475,0.135554,97.2277,97.326733
317,MRO,13.82,N/A,1.154152,94.6535,1.103277,100,0.230977,92.4752,0.160800,98.4158,96.386139
175,FANG,92.40,N/A,1.020851,91.6832,1.098054,99.802,0.217609,90.297,0.203030,99.4059,95.297030
345,NVDA,762.34,N/A,1.030214,92.2772,0.392628,87.7228,0.443723,100,0.231700,99.604,94.900990
218,HES,94.20,N/A,0.816923,85.1485,0.688452,98.6139,0.325178,98.4158,0.109986,94.8515,94.257426
195,FTNT,242.68,N/A,0.753579,81.7822,0.646225,97.8218,0.366035,99.2079,0.132667,96.6337,93.861386
173,EXR,170.07,N/A,0.819167,85.3465,0.453876,92.4752,0.318352,98.2178,0.133550,97.0297,93.267327
174,F,14.82,N/A,1.382492,98.0198,0.655143,98.0198,0.157410,81.3861,0.111639,95.2475,93.168317
249,IRM,47.61,N/A,0.854999,86.7327,0.608193,97.2277,0.293063,96.2376,0.069590,86.5347,91.683168
204,GOOG,2583.10,N/A,0.802931,84.1584,0.456237,93.0693,0.241773,93.0693,0.080422,90.6931,90.247525


In [34]:
hqm_dataframe.sort_values(["HQM Score","One-Month Return Percentile","Three-Month Return Percentile",], ascending = [False,True,False], inplace= True)
hqm_dataframe[:30]

,Ticker,Price,No of shares to Buy,One-Year Price-Return,One-Year Return Percentile,Six-Month Price-Return,Six-Month Return Percentile,Three-Month Price-Return,Three-Month Return Percentile,One-Month Price-Return,One-Month Return Percentile,HQM Score
148,DVN,29.90,N/A,1.333371,97.2277,0.897961,99.604,0.274373,95.2475,0.135554,97.2277,97.326733
317,MRO,13.82,N/A,1.154152,94.6535,1.103277,100,0.230977,92.4752,0.160800,98.4158,96.386139
175,FANG,92.40,N/A,1.020851,91.6832,1.098054,99.802,0.217609,90.297,0.203030,99.4059,95.297030
345,NVDA,762.34,N/A,1.030214,92.2772,0.392628,87.7228,0.443723,100,0.231700,99.604,94.900990
218,HES,94.20,N/A,0.816923,85.1485,0.688452,98.6139,0.325178,98.4158,0.109986,94.8515,94.257426
195,FTNT,242.68,N/A,0.753579,81.7822,0.646225,97.8218,0.366035,99.2079,0.132667,96.6337,93.861386
173,EXR,170.07,N/A,0.819167,85.3465,0.453876,92.4752,0.318352,98.2178,0.133550,97.0297,93.267327
174,F,14.82,N/A,1.382492,98.0198,0.655143,98.0198,0.157410,81.3861,0.111639,95.2475,93.168317
249,IRM,47.61,N/A,0.854999,86.7327,0.608193,97.2277,0.293063,96.2376,0.069590,86.5347,91.683168
204,GOOG,2583.10,N/A,0.802931,84.1584,0.456237,93.0693,0.241773,93.0693,0.080422,90.6931,90.247525
